In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools as it
import functools as ft
from tools import helpers as h
from copy import copy

from umap import UMAP
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

%matplotlib inline

In [2]:
hgnc_mapper = h.fetch_hgnc_mapper()

In [3]:
!ls ../data/grmetrics/

broad_hms_lincs.tsv     genentech_cell_line.tsv lincs_mcf10a.tsv
ctrp.tsv                hms_lincs_density.tsv   mep_lincs.tsv


In [104]:
# initial target list -- not for linear programming
klaeger = pd.read_excel('../data/ref/Klaeger_Science_2017 Supplementary Table 6 Selectivities.xlsx', sheet_name='CATDS target')

# a number of drugs have very similar targets listed in the same row
# we parse to separate these out 
double_drug_targets = klaeger[klaeger['Target'].apply(lambda x: ';' in x)]

# get the first and second gene
first_genes = double_drug_targets['Target'].apply(lambda x: x.split(';')[0])
second_genes = double_drug_targets['Target'].apply(lambda x: x.split(';')[1])

# fix the double targets
repaired_double_targets = double_drug_targets.drop('Target', axis=1).assign(**{'Target':first_genes})
repaired_double_targets = repaired_double_targets.append(double_drug_targets.drop('Target', axis=1).assign(**{'Target':second_genes})) 
klaeger = klaeger.drop(double_drug_targets.index).append(repaired_double_targets, sort=False)

# filter to only hgnc keys
klaeger = klaeger[klaeger.Target.isin(hgnc_mapper.keys())]

# convert to hgnc
klaeger.Target = klaeger.Target.apply(lambda x: hgnc_mapper[x])

# save total drug list information
drug_list = list(sorted(set(klaeger.Drug)))
print(len(drug_list), ' drugs present for our analysis')

# sort columns
klaeger = klaeger[['Target', 'Drug', 'At', 'CATDS']]

klaeger['Drug'] = [x.lower().replace('-', '') for x in klaeger.Drug]
klaeger.head()

222  drugs present for our analysis


,Target,Drug,At,CATDS
0,AURKA,mk5108,0.120921,0.657165
1,CHEK1,pf477736,0.243725,0.733257
2,FLT3,dovitinib,0.265240,0.860803
4,MET,capmatinib,0.337514,1.000000
5,MAPKAPK2,vx702,0.342705,0.743594


In [87]:
klaeger_drugs = set(klaeger.Drug)
print(len(klaeger_drugs), 'drugs in the Klaeger 2017 Science Paper with known targets')

222 drugs in the Klaeger 2017 Science Paper with known targets


In [88]:
datasets = ['broad_hms_lincs.tsv', 'genentech_cell_line.tsv', 
            'lincs_mcf10a.tsv', 'ctrp.tsv',
            'hms_lincs_density.tsv', 'mep_lincs.tsv']

abbrevs = ['broad_hms', 'genentech', 'lincs_mcf', 'ctrp', 'hms_density', 'mep']

datadict = {a:pd.read_csv('../data/grmetrics/'+d, sep='\t') for a,d in list(zip(abbrevs, datasets))}

for name,data in datadict.items():
    print(name,':\tsize',data.shape)
    display(data.head())

broad_hms :	size (642, 12)


,Cell_HMSLID,Cell_Line,Small_Mol_HMSLID,Small_Molecule,GR50,GEC50,GRmax,GRinf,GR_Hill_Coeff,GR_AOC,GR_r2,pval
0,50105-3,BT-20,10001-101-1,Seliciclib,5.677700,14.42560,0.194160,-1.00000,1.17820,0.17548,9.722700e-01,7.689100e-04
1,50105-3,BT-20,10045-101-1,A443654,0.053033,0.23019,-0.492210,-0.63357,0.55758,1.00120,9.874300e-01,1.579200e-04
2,50105-3,BT-20,10133-101-1,Afatinib,inf,0.00000,0.054486,0.68319,0.01000,0.25541,-1.060000e-12,1.290800e-01
3,50105-3,BT-20,10004-101-1,AT-7519,0.204440,0.32038,-0.401780,-0.40563,1.32240,0.86351,9.993600e-01,4.140000e-07
4,50105-3,BT-20,10158-101-1,AZD 5438,0.245910,0.55040,-0.354640,-0.44610,0.79155,0.74810,9.909100e-01,8.250000e-05


genentech :	size (6167, 18)


,Cell_Line,DoublingTime,Tissue,Perturbagen,GR50,GEC50,GRmax,GRinf,GR_Hill_Coeff,GR_AOC,GR_r2,IC50,Emax,AUC,IC_r2,CDC73_del,BCL2_del,PTEN_mut
0,1321N1,23,Brain,bid1870,31.000000,1000.000000,0.81607,0.388820,0.24661,0.066971,0.68658,31.000000,0.739340,0.90021,0.68569,NaN,NaN,NaN
1,1321N1,23,Brain,bortezomib,0.045432,0.063644,-0.63780,-0.640180,2.44540,0.509780,0.99919,0.039072,0.004753,0.63654,0.99806,NaN,NaN,NaN
2,1321N1,23,Brain,crizotinib,1.344200,2.197400,0.70570,-0.999270,2.23430,0.056496,0.67633,1.194200,0.607570,0.91825,0.60390,NaN,NaN,NaN
3,1321N1,23,Brain,docetaxel,0.004362,0.004333,0.00624,0.004647,1.39340,0.482830,0.99431,0.003182,0.116440,0.52438,0.99766,NaN,NaN,NaN
4,1321N1,23,Brain,doxorubicin,0.128300,0.203860,-0.56468,-0.537030,1.57540,0.799740,0.99292,0.098516,0.008452,0.40325,0.99912,NaN,NaN,NaN


lincs_mcf :	size (152, 10)


...

,Cell_Line,Perturbagen,GR50,GEC50,GRmax,GRinf,GR_Hill_Coeff,GR_AOC,GR_r2
0,786O,16-beta-bromoandrosterone,inf,inf,0.93115,0.93115,0.00000,0.020524,0.18804
1,786O,"1S,3R-RSL-3",0.038678,0.047649,-0.89921,-0.91886,5.00000,1.181700,0.99115
2,786O,3-Cl-AHPC,0.952640,1.225600,-0.35674,-0.33655,2.04240,0.515340,0.99784
3,786O,A-804598,inf,1000.000000,0.76195,0.31483,0.39217,0.063627,0.43201
4,786O,AA-COCF3,inf,inf,0.95043,0.95043,0.00000,0.002046,0.24800


hms_density :	size (1008, 22)


,Cell_HMSLID,Cell_Line,Small_Mol_HMSLID,Small_Molecule,Replicate,Density,IC50,EC50,Emax,Einf,...,IC_Curve_Fit,IC_r2,GR50,GEC50,GRmax,GRinf,GR_Hill_Coeff,GR_AOC,GR_Curve_Fit,GR_r2
0,50029-3,MCF7,10024-101-1,NVP-TAE684,1,156.25,0.1519,0.0644,0.3193,0.3262,...,sigmoidal fit: x = Einf + (1-Einf)/(1+ (x/EC50...,0.6193,0.0650,0.0749,-0.1298,-0.1020,1.3170,0.6318,sigmoidal fit: x = Einf + (1-Einf)/(1+ (x/EC50...,0.6437
1,50029-3,MCF7,10024-101-1,NVP-TAE684,1,312.50,0.4509,0.2457,0.2754,0.2733,...,sigmoidal fit: x = Einf + (1-Einf)/(1+ (x/EC50...,0.9794,0.2018,0.2687,-0.2296,-0.2307,1.3254,0.5092,sigmoidal fit: x = Einf + (1-Einf)/(1+ (x/EC50...,0.9843
2,50029-3,MCF7,10024-101-1,NVP-TAE684,1,625.00,0.2192,0.0990,0.2678,0.2470,...,sigmoidal fit: x = Einf + (1-Einf)/(1+ (x/EC50...,0.9380,0.0695,0.1098,-0.2274,-0.2493,0.8837,0.6602,sigmoidal fit: x = Einf + (1-Einf)/(1+ (x/EC50...,0.9391
3,50029-3,MCF7,10024-101-1,NVP-TAE684,1,1250.00,0.1798,0.1022,0.2463,0.2465,...,sigmoidal fit: x = Einf + (1-Einf)/(1+ (x/EC50...,0.9930,0.0842,0.1223,-0.2788,-0.2814,1.1945,0.6383,sigmoidal fit: x = Einf + (1-Einf)/(1+ (x/EC50...,0.9948
4,50029-3,MCF7,10024-101-1,NVP-TAE684,1,2500.00,0.1816,0.1092,0.2284,0.2285,...,sigmoidal fit: x = Einf + (1-Einf)/(1+ (x/EC50...,0.9869,0.0998,0.1392,-0.2196,-0.2296,1.1347,0.5990,sigmoidal fit: x = Einf + (1-Einf)/(1+ (x/EC50...,0.9911


mep :	size (9248, 18)


,Cell_HMSLID,Cell_Line,Clinical_Subtype,Transcriptional_Subtype,Perturbagen_HMSLID,Perturbagen,Perturbagen_Target,Perturbagen_Class,Replicate_ID,Conc_Unit,GR50,GEC50,GRmax,GRinf,GR_Hill_Coeff,GR_AOC,GR_r2,Nominal_Div_Rate
0,51088-1,184A1,NM,Basal,10275-101-1,(Z)-4-Hydroxytamoxifen,ESR1,NaN,9047.132,uM,inf,inf,0.4055,0.4055,0.0000,0.2074,0.3073,0.6985
1,51088-1,184A1,NM,Basal,10236-101-1,17-AAG,HSP90,HSP90,9047.037,uM,0.00930,0.01282,-0.9826,-0.6989,2.7212,1.1760,0.9539,0.5262
2,51088-1,184A1,NM,Basal,10237-101-1,5-DFUR,TYMS,TYMS,12509.143,uM,228.63720,1000.00000,-0.2404,-0.6424,0.5600,0.2642,0.9697,1.3392
3,51088-1,184A1,NM,Basal,10238-101-1,5-FU,TYMS,TYMS,9047.017,uM,48.19500,95.97100,-0.8459,-0.8556,1.4480,0.8094,0.9747,0.8440
4,51088-1,184A1,NM,Basal,10133-101-2,Afatinib,EGFR/HER2,ErbB,12509.165,uM,0.02251,0.33740,-0.9088,-1.0000,0.4058,0.8284,0.9307,1.5022


In [89]:
klaeger_drugs = set([x.lower().replace('-', '') for x in klaeger_drugs])

for name,data in datadict.items():
    try:
        local_drug_set = set(data.Perturbagen)
    except:
        local_drug_set = set(data.Small_Molecule)
        
    local_drug_set = set([x.lower().replace('-','').replace(' ', '') for x in local_drug_set])
    
    print(len(local_drug_set), 'total drugs in', name)
    print(np.round(len(local_drug_set & klaeger_drugs)*100.0/len(klaeger_drugs),3), '% of Klaeger present\n')

107 total drugs in broad_hms
21.622 % of Klaeger present

16 total drugs in genentech
1.351 % of Klaeger present

8 total drugs in lincs_mcf
1.802 % of Klaeger present

545 total drugs in ctrp
29.279 % of Klaeger present

12 total drugs in hms_density
2.703 % of Klaeger present

107 total drugs in mep
7.207 % of Klaeger present



In [90]:
ctrp_dat = datadict['ctrp'][['Cell_Line', 'Perturbagen', 'GR50', 'GRmax']]
ctrp_dat[['GR50', 'GRmax']].describe()

/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3768: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above


,GR50,GRmax
count,65945.0000,65945.000000
mean,NaN,0.014024
std,NaN,0.706069
min,-inf,-0.999990
25%,2.3459,-0.661620
50%,19.8200,-0.039755
75%,NaN,0.729290
max,inf,1.050000


In [91]:
# let's focus on GRmax for now as there are fewer +/- infs
ctrp_dat = ctrp_dat[['Cell_Line', 'Perturbagen', 'GRmax']]

# rename columns
ctrp_dat.columns = ['cellline', 'molecule', 'grmax']

# cut to common drugs
ctrp_dat['molecule'] = ctrp_dat.molecule.apply(lambda x: x.lower().replace('-', '').replace(' ', ''))
ctrp_dat = ctrp_dat[ctrp_dat.molecule.isin(klaeger_drugs)]

display(ctrp_dat.head())
ctrp_dat.describe()

,cellline,molecule,grmax
13,786O,azd1480,0.405350
14,786O,azd4547,-0.841990
17,786O,azd7762,-0.939160
18,786O,azd8055,-0.033395
21,786O,bi2536,-0.802220


,grmax
count,8366.000000
mean,-0.217643
std,0.622463
min,-0.999970
25%,-0.782687
50%,-0.343285
75%,0.247310
max,1.050000


### Init Subclust Drug targets

In [44]:
louv = pd.read_csv('../data/cluster/louvain_clusters.txt', index_col=0, sep='\t')
louv.columns = ['cluster_super']
louv = louv.merge(pd.read_csv('../data/cluster/louvain_small_clusters.txt', index_col=0, sep='\t'), left_index=True, right_index=True)
louv.columns = ['cluster_super', 'cluster_sub']
louv.head()

,cluster_super,cluster_sub
names,,
MST1R,3,11
YES1,3,11
TYRO3,3,11
FGR,3,11
SRC,3,11


In [45]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVR
from sklearn.preprocessing import LabelBinarizer

In [50]:
np.array(louv.cluster_super.unique()).reshape(-1, 1)

array([[3],
       [2],
       [4],
       [7],
       [6],
       [5],
       [8],
       [1],
       [9]])

In [114]:
# convert subclusters to onehot
louv['sub_onehot'] = np.split(OneHotEncoder(categories='auto')\
                              .fit_transform(louv.cluster_sub.values.reshape(len(louv), 1)).toarray(),
                              len(louv))
louv['sub_onehot'] = copy(louv.sub_onehot.apply(lambda x: np.array(x[0])))

# convert kinases to onehot as well
louv['kinase_onehot'] =  np.split(OneHotEncoder(categories='auto')\
                              .fit_transform(louv.index.values.reshape(len(louv), 1)).toarray(),
                              len(louv))
louv['kinase_onehot'] = copy(louv.kinase_onehot.apply(lambda x: np.array(x[0])))
louv.head()

,cluster_super,cluster_sub,sub_onehot,kinase_onehot
names,,,,
MST1R,3,11,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
YES1,3,11,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
TYRO3,3,11,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
FGR,3,11,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
SRC,3,11,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [105]:
# sqrt max - min transform
klaeger['norm_at'] = np.sqrt((klaeger.At-min(klaeger.At))/(max(klaeger.At) - min(klaeger.At)))
klaeger.head()

,Target,Drug,At,CATDS,norm_at
0,AURKA,mk5108,0.120921,0.657165,0.000000
1,CHEK1,pf477736,0.243725,0.733257,0.001112
2,FLT3,dovitinib,0.265240,0.860803,0.001206
4,MET,capmatinib,0.337514,1.000000,0.001477
5,MAPKAPK2,vx702,0.342705,0.743594,0.001495


In [109]:
t = 'azd1480'
ctrp_dat[ctrp_dat.molecule==t]

,cellline,molecule,grmax
13,786O,azd1480,0.405350
1418,A172,azd1480,-0.723990
2390,A673,azd1480,0.268590
2856,ACHN,azd1480,0.433340
3324,ASPC1,azd1480,-0.103110
3799,AU565,azd1480,0.009106
4689,BT20,azd1480,-0.184160
5631,CAKI2,azd1480,0.156930
6516,CAL27,azd1480,-0.601140
8797,CI1,azd1480,1.050000


In [110]:
klaeger[klaeger.Drug == t]

,Target,Drug,At,CATDS,norm_at
1154,PTK2,azd1480,191.364924,0.153965,0.043900
1266,RET,azd1480,237.701575,0.129461,0.048931
1422,AURKA,azd1480,306.075989,0.107222,0.055527
1427,PLK4,azd1480,307.999179,0.106745,0.055701
1778,TNK2,azd1480,515.118931,0.076702,0.072041
1781,FER,azd1480,517.465910,0.076501,0.072205
1981,SLK,azd1480,685.630624,0.065574,0.083115
2028,STK10,azd1480,727.819622,0.063572,0.085635
2087,FGFR1,azd1480,776.837761,0.061487,0.088472
2180,STK3,azd1480,860.221261,0.058411,0.093100


### Fitting 

In [ ]:
#ctrp_dat['kinase_at'] = 
for m in set(ctrp_dat.molecule):
    klaeger_hits = klaeger[klaeger.Drug == m]
    
    klaeger_kinases = klaeger_hits.Target.tolist()
    klaeger_ats = klaeger_hits.norm_at.tolist()
    
    
    
    break